In [1]:
import json
import pandas as pd
import gensim.downloader as api
from sklearn.utils import shuffle
import zipfile
import sys
import requests, io
import re 
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch

In [3]:
def read_json(path: str):
    file = open(path)
    data = json.load(file)
    return pd.DataFrame(data)

In [4]:
data_df = read_json('./documents.json')

In [5]:
print(data_df[data_df["object_type"] == 'rpd'].size)
print(data_df[data_df["object_type"] == 'project'].size)

37360
328


In [58]:
projects = data_df[data_df["object_type"] == 'project']["text"][2:10]
rpds = data_df[data_df["object_type"] == 'rpd']["text"][10:19]

In [59]:
rpds

10    Курс посвящен автоматической обработке и анали...
11    Автономные системы, являющиеся объектом изучен...
12    Дисциплина знакомит с основными понятиями сист...
13    Данный курс знакомит слушателей с основами упр...
14    Содержание дисциплины включает в себя изучение...
15    Целью дисциплины является формирование у студе...
16    Дисциплина посвящена изучению основных принцип...
17    На заре робототехники каждый разработчик робот...
18    Цели дисциплины охватывают ряд направлений, св...
Name: text, dtype: object

In [60]:
model_name = 'DeepPavlov/rubert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [61]:
tokens = {
    'input_ids': [], # list of tensors
    'attention_mask': [] # list of tensors
}

In [62]:
for rpd in rpds:
    new_tokens = tokenizer.encode_plus(rpd, 
                          max_length=128, 
                          truncation=True, 
                          padding='max_length',
                          return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

In [63]:
type(tokens['attention_mask'][0])

torch.Tensor

In [64]:
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [65]:
tokens['input_ids']

tensor([[  101, 33562, 60419,  ...,  3060, 18517,   102],
        [  101, 33576,  2059,  ...,   851, 43399,   102],
        [  101, 33683, 56362,  ..., 15652, 15005,   102],
        ...,
        [  101, 33683, 56362,  ...,  4346, 29700,   102],
        [  101,  2988, 82835,  ...,   128, 75064,   102],
        [  101, 72663, 35135,  ..., 43497,   128,   102]])

In [66]:
tokens['input_ids'].shape

torch.Size([9, 128])

In [67]:
tokens

{'input_ids': tensor([[  101, 33562, 60419,  ...,  3060, 18517,   102],
         [  101, 33576,  2059,  ...,   851, 43399,   102],
         [  101, 33683, 56362,  ..., 15652, 15005,   102],
         ...,
         [  101, 33683, 56362,  ...,  4346, 29700,   102],
         [  101,  2988, 82835,  ...,   128, 75064,   102],
         [  101, 72663, 35135,  ..., 43497,   128,   102]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]])}

In [68]:
outputs = model(**tokens)

In [69]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [70]:
embeddings = outputs.last_hidden_state
embeddings.shape

torch.Size([9, 128, 768])

- 8 предложений;
- 128 токенов;
- 768 размер измерения - размеры скрытого состояния.

In [71]:
attention = tokens['attention_mask']
attention.shape

torch.Size([9, 128])

In [72]:
mask = attention.unsqueeze(-1).expand(embeddings.shape).float()

In [73]:
mask_embedings = embeddings * mask

In [74]:
mask_embedings.shape

torch.Size([9, 128, 768])

In [75]:
summed = torch.sum(mask_embedings, 1) 

In [76]:
summed.shape

torch.Size([9, 768])

In [77]:
counts = torch.clamp(mask.sum(1), min=1e-9)
counts.shape

torch.Size([9, 768])

In [78]:
mean_pooled = summed / counts
mean_pooled.shape

torch.Size([9, 768])

In [79]:
mean_pooled

tensor([[ 0.2456, -0.3694,  0.9801,  ...,  0.7707,  0.7140, -0.6835],
        [-0.0553, -0.5727,  0.6569,  ...,  0.8005,  0.6820, -0.5352],
        [ 0.1983,  0.1160,  0.9993,  ...,  0.5575,  0.9935, -0.6966],
        ...,
        [ 0.0453, -0.6813,  0.8745,  ...,  0.6944,  0.7335, -0.9741],
        [ 0.0356, -0.5139,  0.1714,  ...,  0.7755,  0.5409, -0.6358],
        [ 0.2027, -0.5024,  0.6985,  ...,  0.7258,  0.7510, -1.0071]],
       grad_fn=<DivBackward0>)

In [80]:
from sklearn.metrics.pairwise import cosine_similarity

In [82]:
mean_pooled = mean_pooled.detach().numpy()

cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

array([[0.7775164 , 0.6910979 , 0.7905091 , 0.7696227 , 0.74039704,
        0.74205387, 0.7150265 , 0.7589864 ]], dtype=float32)

In [83]:
rpds[10]

'Курс посвящен автоматической обработке и анализу видеоданных и изображений в интеллектуальных видеокомпьютерных медицинских системах. Основное внимание уделяется методам и алгоритмам цифровой обработки медицинских изображений и видеоданных, методам видеоаналитики, машинного обучения и анализа данных для решения задач обнаружения, сегментации, классификации объектов интереса. Изучаются методы глубокого обучения, а также смарт-технологии визуализации и формирования видеоизображений (создание панорамных изображений, визуализация изображений, сформированных в узких спектральных диапазонах, синтез изображений с учетом персональных особенностей пользователей и др). В рамках курса анализируются структуры и принципы функционирования реальных медицинских видеокомпьютерных систем, а также особенности обработки видеоданных в них. . Целью дисциплины является изучение принципов, методов обработки и анализа медицинских изображений и формирование умений и навыков решения задач видеоаналитики в облас

In [85]:
rpds[11]

'Автономные системы, являющиеся объектом изучения этой дисциплины, - это подвижные роботы, решающие сервисные задачи. Инженеры, разрабатывающие таких роботов, предоставляют интерфейс для низкоуровневого управления системами, поэтому программирование движения по нестандартным траекториям лежит на пользователе. В рамках этой дисциплины рассматриваются способы построения физичной траектории движения, особенности алгоритмов управления автономными системами, а также принципы устройства симуляторов.. Сформировать у обучающихся знания, умения и навыки обоснованного и результативного применения алгоритмов управления автономными системами. Обучить применению существующих алгоритмов теории управления, а также освоению новых методов и алгоритмов управления автономными системами. Дать знания о существующих методах и алгоритмах управления автономными системами. Дать умения разрабатывать алгоритмы управления автономными системами и применять методы компьютерного зрения для управления автономными сис